In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [7]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [11]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [13]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [15]:
dataset=dataset.drop("User ID",axis=1)

In [17]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [19]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [21]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [25]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [27]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [29]:
dependent=dataset[['Purchased']]

In [31]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [33]:
independent.shape

(400, 3)

In [35]:
dependent.shape

(400, 1)

In [37]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [43]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [47]:
from sklearn.svm import SVC

In [61]:
from sklearn.model_selection import GridSearchCV
param_grid={'kernel':['linear','rbf','poly','sigmoid'],'gamma':['auto','scale'],'C':[10,100,1000,2000,3000]}

grid = GridSearchCV(SVC(probability=True), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

grid.fit(x_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [63]:
re=grid.cv_results_
grid_prediction=grid.predict(x_test)

In [65]:
from sklearn. metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)

In [67]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)

C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [75]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("The f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}: 0.17401656314699793


In [79]:
print("The confusion_matrix:\n",cm)

The confusion_matrix:
 [[ 0 79]
 [ 0 41]]


In [81]:
print ("The classification_report:\n",clf_report)

The classification_report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        79
           1       0.34      1.00      0.51        41

    accuracy                           0.34       120
   macro avg       0.17      0.50      0.25       120
weighted avg       0.12      0.34      0.17       120



In [89]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.5

In [91]:
table=pd.DataFrame.from_dict(re)

In [93]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.037663,0.002910,0.020057,0.001439,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
1,0.025131,0.001890,0.018910,0.002408,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
2,0.029999,0.003366,0.020863,0.002891,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,12
3,0.017705,0.002443,0.016449,0.001890,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.769053,0.733523,0.730543,0.753871,0.759910,0.749380,0.014996,32
4,0.026830,0.003023,0.016041,0.001144,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
5,0.023062,0.003917,0.017542,0.001872,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
6,0.026774,0.004067,0.015341,0.001030,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,12
7,0.017946,0.001288,0.017045,0.002634,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.769053,0.733523,0.610390,0.753871,0.759910,0.725349,0.058652,40
8,0.086429,0.008787,0.020967,0.007106,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
9,0.026667,0.002463,0.014842,0.001337,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.855314,0.892857,0.859435,0.929144,0.928571,0.893064,0.031996,3


In [97]:
age_input=int(input("Age:"))
bmi_input=int(input("BMI:"))
children_input=int(input("Children:"))
sex_male_input=int(input("Sex_male 0 or 1:"))
smoker_yes__input=int(input("Smoker_yes 0 or 1:"))

Age: 32
BMI: 43
Children: 2
Sex_male 0 or 1: 0
Smoker_yes 0 or 1: 1


In [99]:
Future_prediction=grid.predict([[age_input,bmi_input,children_input]])

In [101]:
Future_prediction

array([1], dtype=int64)